In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import re

In [2]:
alias = {
    'house': 'SINGLE-FAMILY_HOME',
    'condo': 'APARTMENT,CONDO,COOP',
    'townhouse': 'TOWNHOUSE',
    'multi-family': 'MULTI-FAMILY',
    'land': 'LOT%7CLAND',
    'mobile/manufactured': 'MOBILE%7CMANUFACTURED',
    'other': 'UNKNOWN',
}

def get_buy_webpages(city, 
                     state, 
                     pg_num, 
                     house_type=[
                         alias['house'],
                         alias['multi-family'],
                     ]):
    
    overhead = 'https://trulia.com'
    dangle = 'for_sale'
    tail = f'{house_type[0]},{house_type[1]}_type'
    webpage = f'{overhead}/{dangle}/{city},{state}/'